To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os, re, json
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + "0.0.32.post2" if v == "2.8.0" else "0.0.29.post3"
    !pip install torchmetrics
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

### Google Drive Login


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive



## Load Model

#### Text Completion / Raw Text Training
This is a community notebook collaboration with [Mithex].

We train on `Tiny Stories` (link [here](https://huggingface.co/datasets/roneneldan/TinyStories)) which is a collection of small stories. For example:
```
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun.
Beep was a healthy car because he always had good fuel....
```
Instead of `Alpaca`'s Question Answer format, one only needs 1 column - the `"text"` column. This means you can finetune on any dataset and let your model act as a text completion model, like for novel writing.


In [ ]:
%env UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT
%env SAVE_MODEL_PATH=./outputs/best

env: UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT
env: SAVE_MODEL_PATH=./outputs/best


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it-unsloth-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.8.9: Fast Gemma3 patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

We also add `embed_tokens` and `lm_head` to allow the model to learn out of distribution data.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 69420,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Load Dataset


In [ ]:
!#npm install jsonrepair
!#pip install pythonmonkey

with open('/content/drive/MyDrive/Deep learning/Project/dataset_sorianese.json', 'r', encoding='utf-8') as f:
    data = json.load(f)



train_dataset = data['train']
test_dataset = data['test']
validation_dataset = data['validation']

"""
for i in range(3):
  print(train[i], test[i], validation[i], end='\n\n', sep='\n')

import pythonmonkey, json
jsonrepair = pythonmonkey.require('jsonrepair').jsonrepair
mydataset=[]; malformatted=0
with open('/content/drive/MyDrive/Deep learning/Project/dataset_sorianese1.json', 'w', encoding='utf-8') as f:
  for split_type, ds in enumerate([train, test, validation]):
    for d in ds:
      try:
        d['sentence_text'] = json.loads(d['sentence_text'])['translation']
      except:
        try:
          d['sentence_text'] = json.loads(jsonrepair(d['sentence_text']))['translation']
        except:
            malformatted += 1
      del d['IsLocked'], d['evaluated'], d['split']
    print(malformatted)
  data = {
    'train': train,
    'test': test,
    'validation': validation
  }
  json.dump(data, f, ensure_ascii=False, indent=2)
  """
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(example):
    return { "text" : example["sentence_text"] + EOS_TOKEN }
train_dataset = list(map(formatting_prompts_func, train_dataset))
test_dataset = list(map(formatting_prompts_func, test_dataset))
validation_dataset = list(map(formatting_prompts_func, validation_dataset))
print('Dataset loaded')


Dataset loaded


Print out 5 sentences.

In [ ]:
for row in train_dataset[:5]:
    print("=========================")
    print(row["text"])

Coperchi. Le coperchi sono delle piccole porzioni di dati che vengono memorizzate e utilizzate per migliorare l’esperienza di utilizzo di un sito. Ad esempio possono ricordare tempor<end_of_turn>
Cupertine.<end_of_turn>
Copertine<end_of_turn>
P'i 38 anni suji.<end_of_turn>
Pe' li ssai ott'anni suji.<end_of_turn>


<a name="Train"></a>


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_list(train_dataset[:3000])
test_dataset = Dataset.from_list(test_dataset[:300])
validation_dataset = Dataset.from_list(validation_dataset[:30])

# You can display the structure of the datasets to verify
print("Train dataset structure:")
print(train_dataset)
print("\nValidation dataset structure:")
print(validation_dataset)
print("\nTest dataset structure:")
print(test_dataset)

Train dataset structure:
Dataset({
    features: ['text'],
    num_rows: 3000
})

Validation dataset structure:
Dataset({
    features: ['text'],
    num_rows: 30
})

Test dataset structure:
Dataset({
    features: ['text'],
    num_rows: 300
})


#### Add Callback to calculate Perplexity


## Prepare Trainer
We prepare the train/validation with Hyperparameters.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, EvalPrediction, TrainerCallback
from unsloth import UnslothTrainer, UnslothTrainingArguments
import numpy as np
import torch
import torch.nn.functional as F


In [ ]:

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = validation_dataset,
    eval_dataset = validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    save_total_limit = 1,
    load_best_model_at_end=True,

    args = UnslothTrainingArguments(

        per_device_train_batch_size = 5,
        gradient_accumulation_steps = 1,

        warmup_ratio = 0.1,
        num_train_epochs = 25,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,

        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 42069,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        metric_for_best_model="eval_loss",
        eval_strategy="epoch",
        greater_is_better=False,
    ),

    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)



Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

### Raw Perplexity on Untrained Model

In [ ]:
# Evaluate the currently loaded and trained model on the test dataset
test_results = trainer.evaluate() # (eval_dataset=validation_dataset)
print(test_results)
# The test loss will be in the results
test_loss = test_results["eval_loss"]
print(f"Test Loss: {test_loss}")

# Calculate perplexity from the test loss
import numpy as np
test_perplexity = np.exp(test_loss)
print(f"Test Perplexity: {test_perplexity}")

{'eval_loss': 3.4244158267974854, 'eval_model_preparation_time': 0.0037, 'eval_runtime': 1.0456, 'eval_samples_per_second': 28.693, 'eval_steps_per_second': 7.651}
Test Loss: 3.4244158267974854
Test Perplexity: 30.704702746828904


## Train

By now, we are going to train our model's on the Dataset, at the end of the train we save the best model found during training.

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.816 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()
trainer.save_model(os.getenv('SAVE_MODEL_PATH'))

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 25 | Total steps = 150
O^O/ \_/ \    Batch size per device = 5 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (5 x 1 x 1) = 5
 "-____-"     Trainable parameters = 7,593,984 of 275,692,160 (2.75% trained)
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,3.857700,3.178099,0.003700
2,3.132900,2.551176,0.003700
3,2.271800,1.883749,0.003700
4,2.252700,1.346699,0.003700
5,1.485100,0.966712,0.003700
6,1.250000,0.781920,0.003700
7,1.058500,0.610837,0.003700
8,0.710300,0.499035,0.003700
9,0.690400,0.451746,0.003700
10,0.544500,0.370552,0.003700


In [ ]:
# Access the training log history
log_history = trainer.state.log_history
print(log_history)
# Print the full log history
print(json.dumps(log_history, indent=2))

[{'loss': 3.4038, 'grad_norm': 38.228336334228516, 'learning_rate': 0.0, 'epoch': 0.16666666666666666, 'step': 1}, {'loss': 3.2221, 'grad_norm': 37.67921447753906, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.3333333333333333, 'step': 2}, {'loss': 3.7249, 'grad_norm': 33.35939025878906, 'learning_rate': 6.666666666666667e-06, 'epoch': 0.5, 'step': 3}, {'loss': 2.9717, 'grad_norm': 39.20076370239258, 'learning_rate': 1e-05, 'epoch': 0.6666666666666666, 'step': 4}, {'loss': 3.4002, 'grad_norm': 45.72932052612305, 'learning_rate': 1.3333333333333333e-05, 'epoch': 0.8333333333333334, 'step': 5}, {'loss': 3.8577, 'grad_norm': 37.37812805175781, 'learning_rate': 1.6666666666666667e-05, 'epoch': 1.0, 'step': 6}, {'eval_loss': 3.1780991554260254, 'eval_model_preparation_time': 0.0037, 'eval_runtime': 1.1439, 'eval_samples_per_second': 26.225, 'eval_steps_per_second': 6.993, 'epoch': 1.0, 'step': 6}, {'loss': 3.0879, 'grad_norm': 38.818817138671875, 'learning_rate': 2e-05, 'epoch': 1.166

## Just Evaluation
We are going to evaluate various models on the completion task using the test dataset. We calculate the Perplexity upon the validation dataset.

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

184.3897 seconds used for training.
3.07 minutes used for training.
Peak reserved memory = 1.566 GB.
Peak reserved memory for training = 0.168 GB.
Peak reserved memory % of max memory = 10.623 %.
Peak reserved memory for training % of max memory = 1.14 %.


In [ ]:
# Evaluate the currently loaded and trained model on the test dataset
test_results = trainer.evaluate() # (eval_dataset=validation_dataset)
print(test_results)
# The test loss will be in the results
test_loss = test_results["eval_loss"]
print(f"Test Loss: {test_loss}")

# Calculate perplexity from the test loss
import numpy as np
test_perplexity = np.exp(test_loss)
print(f"Test Perplexity: {test_perplexity}")

{'eval_loss': 0.2506242096424103, 'eval_model_preparation_time': 0.0037, 'eval_runtime': 1.0903, 'eval_samples_per_second': 27.515, 'eval_steps_per_second': 7.337}
Test Loss: 0.2506242096424103
Test Perplexity: 1.2848271679383358


## Inference

### check what you deleted!! 2 cells

<a name="Inference"></a>
### Inference
Let's run the model!

We first will try to see if the model follows the style and understands to write a story that is within the distribution of "Tiny Stories". Ie a story fit for a bed time story most likely.

We select "Once upon a time, in a galaxy, far far away," since it normally is associated with Star Wars.

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

# Before running inference, call `FastLanguageModel.for_inference` first

FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    "Once upon a time, in a galaxy, far far away,"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Once upon a time, in a galaxy, far faraway, there was a little girl named Lily. She loved to 
play with her toys and explore the universe. One day, she found a big, shiny rock. She picked it up and 
put it in her pocket.

Lily went to play with her friends, but she forgot about the rock. When she 
came back home, she realized that she had lost the rock. She was very sad and started to cry.

Her mom 
saw her crying and asked her what was wrong. Lily told her about the rock and how she lost it. Her mom 
said, "Don't worry, we can find it again." They went back to the place where Lily found the rock and 
searched for it. After a while, they found the rock and Lily was very happy. She learned that it's 
important to take care of her things and not to lose them. The end.</s>

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
